In [7]:
# paibox 和 spikingjelly 的推理结果比较，输出为 csv 文件
# 需要修改数据集路径DVS128GESTURE_DATA_DIR
import torch
import numpy as np
import paibox as pb
import csv

import numpy as np
def b(c):
    return np.random.randint(1,127,size=(c,))
def t():
    return np.random.randint(1,127,size=(1,))
def k(out_channels,in_channels,kernel_h=5,kernel_w=5):
    return np.random.randint(1,127,size=(out_channels,in_channels,kernel_h,kernel_w))
def w(in_channels,out_channels):
    return np.random.randint(1,127,size=(in_channels,out_channels))

# PAIBox网络定义
class Conv2d_Net(pb.Network):
    def __init__(self, c):
        super().__init__()

        self.i0 = pb.InputProj(input=None, shape_out=(1, 86, 65))
        self.i1 = pb.InputProj(input=None, shape_out=(1, 86, 65))

        # s = stage, g = group, n = net

        # stage 0 : downsample

        self.s0_g0_n0 = pb.LIF((1, 21, 16), threshold=t(), reset_v=0, tick_wait_start=1) # convpool3x3
        self.s0_g0_conv2d0 = pb.Conv2d(self.i0, self.s0_g0_n0, kernel=k(1,1,7,7), padding=1, stride=4)

        self.s0_g0_n1 = pb.LIF((1, 21, 16), threshold=t(), reset_v=0, tick_wait_start=1) # convpool3x3
        self.s0_g0_conv2d1 = pb.Conv2d(self.i0, self.s0_g0_n1, kernel=k(1,1,7,7), padding=1, stride=4)

        self.s0_g1_n0 = pb.LIF((1, 21, 16), threshold=t(), reset_v=0, tick_wait_start=1) # convpool3x3
        self.s0_g1_conv2d0 = pb.Conv2d(self.i1, self.s0_g1_n0, kernel=k(1,1,7,7), padding=1, stride=4)

        self.s0_g1_n1 = pb.LIF((1, 21, 16), threshold=t(), reset_v=0, tick_wait_start=1) # convpool3x3
        self.s0_g1_conv2d1 = pb.Conv2d(self.i1, self.s0_g1_n1, kernel=k(1,1,7,7), padding=1, stride=4)

        # group shuffle

        self.s1_g0_n0 = self.s0_g0_n0
        self.s1_g0_n1 = self.s0_g1_n0
        self.s1_g1_n0 = self.s0_g0_n1
        self.s1_g1_n1 = self.s0_g1_n1


        self.n1_0_0 = pb.LIF((1, 21, 16), threshold=t(), reset_v=0, tick_wait_start=2) # convpool3x3
        self.conv2d_1_0_0_0 = pb.Conv2d(self.n0_0_0, self.n1_0_0, kernel=k(1,1,7,7), padding=3, stride=1)
        self.conv2d_1_0_0_1 = pb.Conv2d(self.n0_0_1, self.n1_0_0, kernel=k(1,1,7,7), padding=3, stride=1)

        self.n1_0_1 = pb.LIF((1, 21, 16), threshold=t(), reset_v=0, tick_wait_start=2) # convpool3x3
        self.conv2d_1_0_1_0 = pb.Conv2d(self.n0_0_0, self.n1_0_1, kernel=k(1,1,7,7), padding=3, stride=1)
        self.conv2d_1_0_1_1 = pb.Conv2d(self.n0_0_1, self.n1_0_1, kernel=k(1,1,7,7), padding=3, stride=1)

        self.n1_1_0 = pb.LIF((1, 21, 16), threshold=t(), reset_v=0, tick_wait_start=2) # convpool3x3
        self.conv2d_1_1_0_0 = pb.Conv2d(self.n0_1_0, self.n1_1_0, kernel=k(1,1,7,7), padding=3, stride=1)
        self.conv2d_1_1_0_1 = pb.Conv2d(self.n0_1_1, self.n1_1_0, kernel=k(1,1,7,7), padding=3, stride=1)

        self.n1_1_1 = pb.LIF((1, 21, 16), threshold=t(), reset_v=0, tick_wait_start=2) # convpool3x3
        self.conv2d_1_1_1_0 = pb.Conv2d(self.n0_1_0, self.n1_1_1, kernel=k(1,1,7,7), padding=3, stride=1)
        self.conv2d_1_1_1_1 = pb.Conv2d(self.n0_1_1, self.n1_1_1, kernel=k(1,1,7,7), padding=3, stride=1)

        # self.n0_1_0 = pb.LIF((1, 21, 16), threshold=t(), reset_v=0, tick_wait_start=2) # conv3x3
        # self.conv2d_1_0 = pb.Conv2d(self.n0_0_0, self.n0_1_0, kernel=k(2,2), padding=2, stride=1)

        # self.n0_1 = pb.LIF((c, 28, 21), threshold=t(), reset_v=0, tick_wait_start=2) # conv3x3
        # self.conv2d_1_1 = pb.Conv2d(self.n0_1, self.n1_1, kernel=k(c,c), padding=2, stride=1)

        # self.n2 = pb.LIF((c*4, 14, 11), threshold=t(), reset_v=0, tick_wait_start=3) # convpool3x3
        # # self.conv2d_2 = pb.Conv2d(self.n1, self.n2, kernel=k(c*4,c*2), padding=2, stride=2)
        # self.conv2d_2_0 = pb.Conv2d(self.n1_0, self.n2, kernel=k(c*4,c), padding=2, stride=2)
        # self.conv2d_2_1 = pb.Conv2d(self.n1_1, self.n2, kernel=k(c*4,c), padding=2, stride=2)

        # self.n3 = pb.LIF((c*4, 14, 11), threshold=t(), reset_v=0, tick_wait_start=4) # conv3x3
        # self.conv2d_3 = pb.Conv2d(self.n2, self.n3, kernel=k(c*4,c*4), padding=2, stride=1)

        self.n10 = pb.LIF(c*8 * 8 * 8, threshold=t(), reset_v=0, tick_wait_start=5) # fc
        self.fc_0_0_0 = pb.FullConn(self.n1_0_0, self.n10, conn_type=pb.SynConnType.All2All, weights=w(1*21*16,c*8*8*8))
        self.fc_0_0_1 = pb.FullConn(self.n1_0_1, self.n10, conn_type=pb.SynConnType.All2All, weights=w(1*21*16,c*8*8*8))
        self.fc_0_0_2 = pb.FullConn(self.n0_1_0, self.n10, conn_type=pb.SynConnType.All2All, weights=w(1*21*16,c*8*8*8))
        self.fc_0_0_3 = pb.FullConn(self.n0_1_1, self.n10, conn_type=pb.SynConnType.All2All, weights=w(1*21*16,c*8*8*8))



        self.n11 = pb.LIF(c*8 * 4 * 4, threshold=t(), reset_v=0, tick_wait_start=6) # fc
        self.fc_1 = pb.FullConn(self.n10, self.n11, conn_type=pb.SynConnType.All2All, weights=w(c*8*8*8,c*8*4*4))

        self.n12 = pb.LIF(80, threshold=t(), reset_v=0, tick_wait_start=7) # fc
        self.fc_2 = pb.FullConn(self.n11, self.n12, conn_type=pb.SynConnType.All2All, weights=w(c*8*4*4,80))

        self.probe1 = pb.Probe(self.n12, "spike")

C = 2

# PAIBox仿真器
pb_net = Conv2d_Net(C)
sim = pb.Simulator(pb_net)

# test(test_num=50)
pb.BACKEND_CONFIG.target_chip_addr = (0, 0)

In [8]:
mapper = pb.Mapper()

mapper.build(pb_net)

# Core estimate only
graph_info = mapper.compile(
    weight_bit_optimization=True, grouping_optim_target="both", core_estimate_only=True
)
 
# #N of cores required
print("Core required:", graph_info["n_core_required"])

# Clear all the results
mapper.clear()

Core required: 250


In [3]:
mapper = pb.Mapper()

mapper.build(pb_net)

graph_info = mapper.compile(
    weight_bit_optimization=True, grouping_optim_target="both"
)

# #N of cores required
print("Core required:", graph_info["n_core_required"])

Core required: 897


In [ ]:
mapper.export(
    write_to_file=True, fp="./debug", format="npy", export_core_params=False
)

# Clear all the results
mapper.clear()